In [2]:
import os
from webbrowser import get
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from collections import namedtuple
from tqdm.notebook import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
import shutil

def get_carla_trainId():
    
    CarlaClass = namedtuple('CarlaClass', ['carla_name', 'carla_id', 'carla_color', 'train_id', 'cs_id', 'cs_color'])
    classes = [
        CarlaClass('unlabeled',       0,  (0, 0, 0),       255, 0,  (0, 0, 0)),
        CarlaClass('building',        1,  (70, 70, 70),    0,   11, (70, 70, 70)),
        CarlaClass('fence',           2,  (100, 40, 40),   1,   13, (190, 153, 153)),
        CarlaClass('other',           3,  (55, 90, 80),    255, 0,  (0, 0, 0)),
        CarlaClass('pedestrian',      4,  (220, 20, 60),   2,   24, (220, 20, 60)),
        CarlaClass('pole',            5,  (153, 153, 153), 3,   17, (153, 153, 153)),
        CarlaClass('road line',       6,  (157, 234, 50),  4,   7,  (128, 64, 128)),
        CarlaClass('road',            7,  (128, 64, 128),  4,   7,  (128, 64, 128)),
        CarlaClass('sidewalk',        8,  (244, 35, 232),  5,   8,  (244, 35, 232)),
        CarlaClass('vegetation',      9,  (107, 142, 35),  6,   21, (107, 142, 35)),
        CarlaClass('vehicle',         10, (0, 0, 142),     7,   26, (0, 0, 142)),
        CarlaClass('wall',            11, (102, 102, 156), 8,   12, (102, 102, 156)),
        CarlaClass('traffic sign',    12, (220, 220, 0),   9,   20, (220, 220, 0)),
        CarlaClass('sky',             13, (70, 130, 180),  10,  23, (70, 130, 180)),
        CarlaClass('ground',          14, (81, 0, 81),     11,  6,  (81, 0, 81)),
        CarlaClass('bridge',          15, (150, 100, 100), 12,  15, (150, 100, 100)),
        CarlaClass('rail track',      16, (230, 150, 140), 13,  0,  (0, 0, 0)),
        CarlaClass('guard rail',      17, (180, 165, 180), 14,  14, (180, 165, 180)),
        CarlaClass('traffic light',   18, (250, 170, 30),  15,  19, (250, 170, 30)),
        CarlaClass('static',          19, (110, 190, 160), 255, 4,  (0, 0, 0)),
        CarlaClass('dynamic',         20, (170, 120, 50),  255, 5,  (111, 74, 0)),
        CarlaClass('water',           21, (45, 60, 150),   255, 0,  (0, 0, 0)),
        CarlaClass('terrain',         22, (145, 170, 100), 16,  22, (152, 251, 152)),
        CarlaClass('general anomaly', 23, (236, 236, 236), 255, 0,  (0, 0, 0)),
    ]

    return classes



In [2]:
carla_map = get_carla_trainId()

color_map = [list(c.carla_color) for c in carla_map]
color_map

[[0, 0, 0],
 [70, 70, 70],
 [100, 40, 40],
 [55, 90, 80],
 [220, 20, 60],
 [153, 153, 153],
 [157, 234, 50],
 [128, 64, 128],
 [244, 35, 232],
 [107, 142, 35],
 [0, 0, 142],
 [102, 102, 156],
 [220, 220, 0],
 [70, 130, 180],
 [81, 0, 81],
 [150, 100, 100],
 [230, 150, 140],
 [180, 165, 180],
 [250, 170, 30],
 [110, 190, 160],
 [170, 120, 50],
 [45, 60, 150],
 [145, 170, 100],
 [236, 236, 236]]

In [3]:
delivery_dir = "/home/gaoha/epe/Delivery/Carla2DarkZurich_v2"
dst_folder_name = "Carla2DarkZurich"

dst_dir = f"/home/chenht/datasets/{dst_folder_name}"

if not os.path.isdir(dst_dir):
    os.mkdir(dst_dir)

if not os.path.isdir(os.path.join(dst_dir, "train")):
    os.mkdir(os.path.join(dst_dir, "train"))

if not os.path.isdir(os.path.join(dst_dir, "val")):
    os.mkdir(os.path.join(dst_dir, "val"))

if not os.path.isdir(os.path.join(dst_dir, "train", "semantic")):
    os.mkdir(os.path.join(dst_dir, "train", "semantic"))
    
if not os.path.isdir(os.path.join(dst_dir, "train", "rgb")):
    os.mkdir(os.path.join(dst_dir, "train", "rgb"))
    
if not os.path.isdir(os.path.join(dst_dir, "val", "semantic")):
    os.mkdir(os.path.join(dst_dir, "val", "semantic"))
    
if not os.path.isdir(os.path.join(dst_dir, "val", "rgb")):
    os.mkdir(os.path.join(dst_dir, "val", "rgb"))

filenames = os.listdir(delivery_dir)

filenames = [filename.split('_rgb')[0] for filename in filenames if 'rgb' in filename]

filename_train, filename_val = train_test_split(filenames, test_size=0.3, random_state=42)
print(len(filename_train), len(filename_val))

5786 2481


In [11]:
for filename in tqdm(filename_train):
    label = np.array(Image.open(os.path.join(delivery_dir, filename + "_semantic.png")))
    height, width, _ = label.shape
    label = label.reshape(height * width, 3)
    new_label = np.zeros((height * width, 1))
    for i in range(height * width):
        new_label[i] = color_map.index(label[i].tolist())
    new_label = new_label.reshape(height, width).astype('uint8')
    Image.fromarray(new_label).save(os.path.join(dst_dir, 'train/semantic', filename + '_semantic.png'))
    
    shutil.copyfile(os.path.join(delivery_dir, filename + '_rgb.png'), os.path.join(dst_dir, 'train/rgb', filename + '_rgb.png'))

    
for filename in tqdm(filename_val):
    label = np.array(Image.open(os.path.join(delivery_dir, filename + "_semantic.png")))
    height, width, _ = label.shape
    label = label.reshape(height * width, 3)
    new_label = np.zeros((height * width, 1))
    for i in range(height * width):
        new_label[i] = color_map.index(label[i].tolist())
    new_label = new_label.reshape(height, width).astype('uint8')
    Image.fromarray(new_label).save(os.path.join(dst_dir, 'val/semantic', filename + '_semantic.png'))
    
    shutil.copyfile(os.path.join(delivery_dir, filename + '_rgb.png'), os.path.join(dst_dir, 'val/rgb', filename + '_rgb.png'))


  0%|          | 0/5786 [00:00<?, ?it/s]

In [12]:
from tqdm.notebook import tqdm_notebook as tqdm
c = 0
dir = "/home/chenht/datasets/NightLab/train/image/"
filenames = [dir + filename for filename in os.listdir(dir)]
# for filename in tqdm(filenames, total=len(filenames)):
#     if np.amax(np.array(Image.open(filename))) > 33:
#         c += 1
# print(c)
    
len(filenames)

  0%|          | 0/1299 [00:00<?, ?it/s]

0


(8301164,)